# Runge-Kutta Benchmarks on Linear ODEs

This notebook is to document benchmarks between the Runge-Kutta solvers implemented in Julia. These are the implementations native to OrdinaryDiffEq.jl, ODE.jl, and those the classic Hairer methods provided by ODEInterface.jl. We will use OrdinaryDiffEq.jl's benchmarking framework for performing these tests. The purpose of this notebook is to complement the work-precision diagrams by showing how the same algorithm (Runge-Kutta Order 4) is many times faster in OrdinaryDiffEq.jl than in ODE.jl, and that OrdinaryDiffEq.jl specializes on scalar problems for more efficiency gains.

First, let's define our problems:

In [10]:
using DifferentialEquations, ODE, ODEInterfaceDiffEq
tspan = (0.0,1.0)
# Linear ODE
f = (t,u) -> (1.01*u)
(::typeof(f))(::Type{Val{:analytic}},t,u₀) = u₀*exp(1.01*t)
probnum = ODEProblem(f,1/2,tspan)

# 2D Linear ODE
f = (t,u,du) -> begin
  for i in 1:length(u)
    du[i] = 1.01*u[i]
  end
end
(::typeof(f))(::Type{Val{:analytic}},t,u₀) = u₀*exp(1.01*t)
prob = ODEProblem(f,rand(100,100),tspan)
using Plots; gr()

Plots.GRBackend()

Here, `probnum` is a simply the 1-dimensional linear ODE, and `prob` is a 100x100 matrix of linear ODEs.

Although  the linear ODE may be the most "basic" case, the linear ODE on [0,10] has some interesting qualities. Since it rises so fast in the end, most of the error is accumulated there. However, error from earlier is compounded in later stages. Therefore, in order for an algorithm to achieve low error, it has to be able to evenly distribute its error (in percentage) about the whole problem. Thus it is a good test of the ability for the adaptivity algorithm to control error globally. Also, the small time it takes for the actual function calculations better highlights the differences between implementations. Lastly, the steep rise in the end can be a problem for methods trying to scrape by with higher error but faster speeds. This means that lower order methods tend to get trapped, but too high of an order simply costs too much. Who will do best? Let's see what we get!

## First, A Choice

Note that DifferentialEquations.jl by default stores more information than the other algorithms. This is for the continuous output that is allowed via `sol(t)`. To make the tests fair, we turn this off for the tests.

## RK4

To use the suite, we simply define the setups we want to run on a problem, and then call the shootout function. Let's start by testing the RK4 implementations on probnum. RK4 algorithms are fixed timestep, and so by choosing the same timestep, we should arrive at around the same error (sans numerical truncation), meaning this is a good test of how efficient the most basic implementaions are. Since RK4 is a relatively simple algorithm, the differences shouldn't be too dramatic. Here we will test DifferentialEquations' native `RK4` vs the implementation `rk4` from ODE.jl.

In [18]:
setups = [Dict(:alg=>RK4());Dict(:alg=>RK4(),:dense=>false);Dict(:alg=>RK4(),:dense=>false,:save_everystep=>false);Dict(:alg=>ode4())]
names = ["RK4 Continuous";"RK4 Non-Dense";"RK4 Save End";"ODE.jl rk4"]
shoot = Shootout(probnum,setups;dt=1/2^(6),names=names);

Here, we define:

$$ Efficiency = \frac{1}{Error \times Time} $$

Thus if an algorithm results in less error or faster runtimes, it's calculated as more efficient. These results show that DifferentialEquations' `:RK4` is 100x as efficient as ODE.jl's `:ode4`. We can get more information by printing the results:

In [19]:
@show shoot.times # Times
@show shoot.errors # Errors
@show shoot.effs # Efficiencies
@show shoot.effratios[shoot.bestidx,:] # Efficiencies

shoot.times = [9.74149e-5,8.28862e-5,6.77531e-5,0.000142153]
shoot.errors = [7.07299e-10,7.07299e-10,7.07299e-10,0.00523784]
shoot.effs = [7.25674e11,8.52874e11,1.04337e12,67152.3]
shoot.effratios[shoot.bestidx,:] = [1.43779,1.22336,1.0,1.55373e7]


4-element Array{Float64,1}:
 1.43779  
 1.22336  
 1.0      
 1.55373e7

This shows that the resulting error was essentialy the same, but `:RK4` ran faster ~3.5x faster. This shows how the DifferentialEquations.jl algorithms are well-optimized, and even the non-multithreaded versions can beat other other Julia implmentations. This is by default a mean of 20 runs and is thus quite stable result. A plot recipe is provided to plot the efficiencies together:

In [20]:
plot(shoot)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 RK4 Continuous 
 
 
 RK4 Non-Dense 
 
 
 RK4 Save End 
 
 
 ODE.jl rk4 
 
 
 0 
 
 
 200000000000 
 
 
 400000000000 
 
 
 600000000000 
 
 
 800000000000 
 
 
 1000000000000 
 
 
 Algorithms 
 
 
 Efficiency

Next we test the difference between the implementations on the larger problem.

In [22]:
setups = [Dict(:alg=>RK4());Dict(:alg=>RK4(),:dense=>false);Dict(:alg=>RK4(),:dense=>false,:save_everystep=>false);Dict(:alg=>ode4())]
names = ["RK4 Continuous";"RK4 Non-Dense";"RK4 Save End";"ode4"]
shoot = Shootout(prob,setups;dt=1/2^(6),names=names)
println(shoot.effratios[shoot.bestidx,:])
plot(shoot)

[10.6175,8.34658,1.0,1.04253e6]


<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 RK4 Continuous 
 
 
 RK4 Non-Dense 
 
 
 RK4 Save End 
 
 
 ode4 
 
 
 0 
 
 
 5000000000 
 
 
 10000000000 
 
 
 15000000000 
 
 
 Algorithms 
 
 
 Efficiency

As the cost of function calculations increases, the implementations, they are still calculating essentially the same thing, but the efficiency gap grows. With 100x100 matrices, OrdinaryDiffEq.jl in "fast-mode" is 1e6x faster than ODE.jl, and still 10x faster and 8x faster with the full timeseries and with continuous (interpolated) output respectively. This is a huge differece which shows that the OrdinaryDiffEq algorithms have a much better implementation.

### Results on Numbers

Now we test the results on "numbers", i.e. ODEs in one variable. This won't even be fair because OrdinaryDiffEq.jl provides special dispatches on number while the other wrapped packages treat them as size-1 arrays (this is by the design of their packages). `rk45` is from ODE.jl and `dopri5` is the classic Hairer Fortran method.

In [29]:
setups = [Dict(:alg=>DP5())
          Dict(:abstol=>1e-3,:reltol=>1e-6,:alg=>ode45()); # Fix ODE to be normal
          Dict(:alg=>dopri5())]
shoot = Shootout(probnum,setups)
println(shoot.times)
println(shoot.errors)
println(shoot.effratios[shoot.bestidx,:])
plot(shoot)

[0.0001122,0.000245091,0.000283293]
[2.82981e-6,1.54217e-6,2.82981e-6]
[1.0,1.19044,2.52488]


In this case, `DP5` is about 2x times as fast as the other algorithms and still achieves less error, making it the clear winner. This is even without continuous output turned off. More features, faster, less error, etc.

(Note: We had to fix ODE.jl's tolerances to match those of OrdinaryDiffEq.jl and the Hairer algorithms which have the same defaults. This is because you can artificially make an algorithm "more efficient" by using lower tolerances becuase the order of the error higher than one, and thus efficiency is always higher at lower tolerances)

### Conclusions

As you can see, a lot of time has been spent on OrdinaryDiffEq.jl's non-stiff solvers are not only feature-filled, but also very efficient. ODEs in one variable are special-cased in OrdinaryDiffEq.jl's algorithms to give maximal efficiency. All of the methods can work on arbitrary-sized arrays, and the methods are fully devectorized and utilize caching to have nearly zero allocations during the core loop. Tweaks on stepsize stabilization methods are also included, resulting in an even more efficient stepsize method. Thus even in the odd cases (non-adaptive algorithms, scalar ODEs, etc.) the OrdinaryDiffEq.jl algorithms are much faster than the ODE.jl and classic Fortran algorithms.